In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np

In [2]:
import pymove as pm
from pymove.visualization import folium as f, matplotlib as mpl
from pymove.utils import visual

In [3]:
import pandas as pd
import pymove as pm
from pymove import folium, MoveDataFrame
from pymove.query import query

In [4]:
move_df = pd.read_csv("atlantic.csv", encoding= 'unicode_escape')

In [5]:
move_df = pm.conversions.lat_and_lon_decimal_degrees_to_decimal(
    move_df, latitude='Latitude', longitude='Longitude'
)

def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
move_df['Datetime'] = move_df.apply(convert_to_datetime, axis=1)

move_df[['ID', 'Name', 'Latitude', 'Longitude', 'Datetime']].head()

,ID,Name,Latitude,Longitude,Datetime
0,AL011851,UNNAMED,28.0,-94.8,1851-06-25 00:00:00
1,AL011851,UNNAMED,28.0,-95.4,1851-06-25 06:00:00
2,AL011851,UNNAMED,28.0,-96.0,1851-06-25 12:00:00
3,AL011851,UNNAMED,28.1,-96.5,1851-06-25 18:00:00
4,AL011851,UNNAMED,28.2,-96.8,1851-06-25 21:00:00


In [6]:
#Converting the pandas dataframe to pymove's MoveDataFrame
move_pm = MoveDataFrame(
    data=move_df, latitude='Latitude', longitude='Longitude',datetime='Datetime', traj_id='Name'
)
#print(type(move_pm))
#move_pm.head()

In [17]:
move_df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Datetime
0,AL011851,UNNAMED,18510625,0,,HU,28.0,-94.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 00:00:00
1,AL011851,UNNAMED,18510625,600,,HU,28.0,-95.4,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 06:00:00
2,AL011851,UNNAMED,18510625,1200,,HU,28.0,-96.0,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 12:00:00
3,AL011851,UNNAMED,18510625,1800,,HU,28.1,-96.5,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 18:00:00
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2,-96.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,,EX,41.3,-50.4,55,981,...,120,120,120,60,0,0,0,0,0,2015-11-12 12:00:00
49101,AL122015,KATE,20151112,1800,,EX,41.9,-49.9,55,983,...,120,120,120,60,0,0,0,0,0,2015-11-12 18:00:00
49102,AL122015,KATE,20151113,0,,EX,41.5,-49.2,50,985,...,220,120,120,60,0,0,0,0,0,2015-11-13 00:00:00
49103,AL122015,KATE,20151113,600,,EX,40.8,-47.5,45,985,...,220,0,0,0,0,0,0,0,0,2015-11-13 06:00:00


In [7]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [8]:
def colour_ge_value(value, comparison):
    if value > comparison:
        return 'color: red'
    else:
        return 'color: black'

In [47]:
output = widgets.Output()
plot_output = widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(move_df.ID), description = 'ID:')
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(move_df.Time), description = 'Time:')
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1, description = 'Number:')


def common_filtering(Id, time, num):
    output.clear_output()
    
    if (Id == ALL) & (time == ALL):
        common_filter = move_df
    elif (Id == ALL):
        common_filter = move_df[move_df.Time == time]
    elif (time == ALL):
        common_filter = move_df[move_df.ID == Id]
    else:
        common_filter = move_df[(move_df.ID == Id) & (move_df.Time == time)]
    
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num), subset=['Latitude', 'Time']))
        

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,
                     bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, 
                     bounded_num.value)
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)

    

dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

display(output)
display(plot_output)

Dropdown(description='ID:', options=('ALL', 'AL011851', 'AL011852', 'AL011853', 'AL011854', 'AL011855', 'AL011…

Dropdown(description='Time:', options=('ALL', 0, 30, 45, 100, 130, 200, 230, 245, 300, 315, 330, 340, 400, 410…

BoundedFloatText(value=5.0, description='Number:', max=100000.0, step=1.0)

Output()

Output()

Output()